# SegmentId Model Prototype

Explore the `SegmentId` Pydantic model design and `SheetManager` enhancements.


## Import


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from pydantic import BaseModel

from snap_fit.config.types import EdgePos
from snap_fit.params.snap_fit_params import get_snap_fit_params

## Params and config


In [ ]:
project_name_params = get_snap_fit_params()
rprint(project_name_params)

## Define SegmentId model


In [ ]:
class SegmentId(BaseModel, frozen=True):
    """Unique identifier for a segment across sheets/pieces/edges.

    Frozen for hashability (can be used in sets/dicts).
    """

    sheet_id: str
    piece_id: int
    edge_pos: EdgePos

    def __str__(self) -> str:
        """Human-readable string representation."""
        return f"{self.sheet_id}:{self.piece_id}:{self.edge_pos.value}"

    def __repr__(self) -> str:
        """Detailed repr for debugging."""
        return f"SegmentId({self.sheet_id!r}, {self.piece_id}, {self.edge_pos!r})"

    @property
    def as_tuple(self) -> tuple[str, int, EdgePos]:
        """Return as tuple for unpacking compatibility."""
        return (self.sheet_id, self.piece_id, self.edge_pos)

## Test SegmentId basics


In [ ]:
# Create some segment IDs
seg1 = SegmentId(sheet_id="sheet_01", piece_id=0, edge_pos=EdgePos.LEFT)
seg2 = SegmentId(sheet_id="sheet_01", piece_id=0, edge_pos=EdgePos.RIGHT)
seg3 = SegmentId(sheet_id="sheet_01", piece_id=1, edge_pos=EdgePos.LEFT)

rprint(f"seg1: {seg1}")
rprint(f"seg2: {seg2}")
rprint(f"repr: {seg1!r}")
rprint(f"as_tuple: {seg1.as_tuple}")

In [ ]:
# Test hashability (frozen=True)
seg_set = {seg1, seg2, seg3}
rprint(f"Set of segments: {seg_set}")

# Test equality
seg1_copy = SegmentId(sheet_id="sheet_01", piece_id=0, edge_pos=EdgePos.LEFT)
rprint(f"seg1 == seg1_copy: {seg1 == seg1_copy}")
rprint(f"seg1 in seg_set: {seg1 in seg_set}")

# Test as dict key
seg_dict = {seg1: "some_data", seg2: "other_data"}
rprint(f"seg_dict[seg1]: {seg_dict[seg1]}")

In [ ]:
# Test serialization
rprint(f"model_dump: {seg1.model_dump()}")
rprint(f"model_dump_json: {seg1.model_dump_json()}")

# Test deserialization
seg1_json = seg1.model_dump_json()
seg1_restored = SegmentId.model_validate_json(seg1_json)
rprint(f"Restored: {seg1_restored}")
rprint(f"Equal after restore: {seg1 == seg1_restored}")

## Prototype SheetManager methods

Sketch out how the new methods would work.


In [ ]:
# Prototype helper functions that will become SheetManager methods


def get_segment_ids_all(sheets: dict[str, "Sheet"]) -> list[SegmentId]:
    """Get all segment IDs from all sheets."""
    segment_ids: list[SegmentId] = []
    for sheet_id, sheet in sheets.items():
        for piece in sheet.pieces:
            for edge_pos in EdgePos:
                segment_ids.append(
                    SegmentId(
                        sheet_id=sheet_id, piece_id=piece.piece_id, edge_pos=edge_pos
                    )
                )
    return segment_ids


def get_segment_ids_other_pieces(
    sheets: dict[str, "Sheet"],
    seg_id: SegmentId,
) -> list[SegmentId]:
    """Get segment IDs from all pieces except the one in seg_id."""
    all_ids = get_segment_ids_all(sheets)
    return [
        sid
        for sid in all_ids
        if not (sid.sheet_id == seg_id.sheet_id and sid.piece_id == seg_id.piece_id)
    ]


rprint("Functions defined successfully")

In [ ]:
# Simulate a sheets dict to test the functions
# In reality, Sheet objects would come from SheetManager


class MockPiece:
    def __init__(self, piece_id: int):
        self.piece_id = piece_id


class MockSheet:
    def __init__(self, pieces: list[MockPiece]):
        self.pieces = pieces


# Create mock data: 2 sheets, each with 2 pieces
mock_sheets = {
    "sheet_a": MockSheet([MockPiece(0), MockPiece(1)]),
    "sheet_b": MockSheet([MockPiece(0), MockPiece(1)]),
}

all_ids = get_segment_ids_all(mock_sheets)
rprint(f"Total segment IDs: {len(all_ids)}")  # 2 sheets * 2 pieces * 4 edges = 16
for sid in all_ids[:6]:
    rprint(f"  {sid}")

In [ ]:
# Test get_segment_ids_other_pieces
test_seg = SegmentId(sheet_id="sheet_a", piece_id=0, edge_pos=EdgePos.LEFT)
other_ids = get_segment_ids_other_pieces(mock_sheets, test_seg)

rprint(f"Query segment: {test_seg}")
rprint(
    f"Other pieces segments: {len(other_ids)}"
)  # 16 - 4 = 12 (excluding piece 0 from sheet_a)

# Verify piece 0 from sheet_a is excluded
excluded = [sid for sid in other_ids if sid.sheet_id == "sheet_a" and sid.piece_id == 0]
rprint(f"Excluded segments from same piece: {len(excluded)} (should be 0)")